In this notebook, I’ll build on the facilities provided by the Instrument class (that is, its ability to
detect changes in its inputs and recalculate accordingly) to show how to calculate numerical Greeks
when the engine doesn’t provide them.

In [1]:
import QuantLib as ql

In [2]:
today = ql.Date(8, ql.October, 2014)
ql.Settings.instance().evaluationDate = today

In [3]:
# Knock-in barrier option
option = ql.BarrierOption(
    ql.Barrier.UpIn,
    120.0, # barrier
    0.0, # rebate
    ql.PlainVanillaPayoff(ql.Option.Call, 100.0),
    ql.EuropeanExercise(ql.Date(8, ql.January, 2015))
)

For the purpose of this example, the market data are the underlying value, the risk-free rate and the
volatility. We wrap them in quotes, so that the instrument will be notified of any changes

In [4]:
u = ql.SimpleQuote(100.0)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.20)

In [5]:
riskFreeCurve = ql.FlatForward(0, ql.TARGET(),
ql.QuoteHandle(r), ql.Actual360())
volatility = ql.BlackConstantVol(0, ql.TARGET(),
ql.QuoteHandle(sigma), ql.Actual360())

process = ql.BlackScholesProcess(ql.QuoteHandle(u),
ql.YieldTermStructureHandle(riskFreeCurve),
ql.BlackVolTermStructureHandle(volatility))

In [6]:
option.setPricingEngine(ql.AnalyticBarrierEngine(process))

In [7]:
print(option.NPV())

1.3657980739109867


In [8]:
print(option.delta())

RuntimeError: delta not provided

The analytical engine doesn't provide a delta. So an error is raised.

## Numerical calculation

$\Delta = \frac{P(u_0+h)-P(u_0-h)}{2h} \quad   \Gamma = \frac{P(u_0+h)-2P(u_0)+P(u_0-h)}{h^2} $

where $P(u)$ is the price of the option for a given value of the underlying $u$.

In [9]:
u0 = u.value() ; h = 0.01
P0 = option.NPV()
print(P0)

1.3657980739109867


In [10]:
u.setValue(u0+h)
P_plus = option.NPV(); print(P_plus)

1.3688112201958078


In [11]:
u.setValue(u0-h)
P_minus = option.NPV(); print(P_minus)

1.3627900998610203


In [12]:
u.setValue(u0)
Delta = (P_plus - P_minus)/(2*h)
Gamma = (P_plus - 2*P0 + P_minus)/(h*h)
print(Delta)
print(Gamma)

0.3010560167393761
0.05172234854633473


This can be done for any greek. We can use either the two-sided formula or the one-sided formula to minimize calculations.

The approach for the Theta is a bit different, although it still relies on the fact that the option reacts
to the change in the market data. The problem is that we don’t have the time to maturity available
as a quote, as was the case for the other quantities. Instead, since we set up the term structures
so that they move with the evaluation date, we just have to set it to tomorrow’s date to get the
corresponding option value:

In [22]:
instance = ql.Settings.instance()
instance.evaluationDate = today + 1
P1 = option.NPV()
h = 1.0/365
Theta = (P1-P0)/h ; print(Theta)

-10.770888399441302


Date(9,10,2014)